In [2]:
import numpy as np
import pandas as pd
import json

import open3d as o3d
import trimesh
import shapely
import rtree

from scipy.spatial import cKDTree

from auxiliary import values as v
from auxiliary.data.dataset_ht import find_group, HtDataset
from auxiliary.utils.colors import bcolors as bc

In [3]:
folder = v.data_path + 'ATLAS/myocardium/'
folder_out = v.data_path + 'Landmarks/ATLAS/'

In [4]:
# gr = 1
# mesh = o3d.io.read_triangle_mesh(folder + f'ATLAS_Gr{gr}.ply')
# mesh.compute_vertex_normals()
# 
# vis = o3d.visualization.VisualizerWithVertexSelection()
# vis.create_window()
# vis.add_geometry(mesh)
# 
# print(
#     f'INSTRUCTIONS:'
#     f'\n\t-Press {bc.OKBLUE}Shift + Left Click{bc.ENDC} to select a vertex.'
#     f'\n\t-Press {bc.OKBLUE}Q{bc.ENDC} to quit.'
# )
# 
# vis.run()
# vis.destroy_window()
# 
# selected_vertices_pickle = vis.get_picked_points()
# selected_vertices = [p.index for p in selected_vertices_pickle]
# selected_coords = np.asarray(mesh.vertices)[selected_vertices]
# 
# # Save points as json
# selected_coords = np.round(selected_coords, 3)
# selected_dict = {idx+1: v.tolist() for idx, v in enumerate(selected_coords[::-1])}
# 
# selected_dict_pinned = {idx+1: v for idx, v in enumerate(selected_vertices[::-1])}
# 
# print(f'Selected vertices:')
# print(selected_dict)
# print(selected_dict_pinned)

In [43]:
import open3d as o3d
import numpy as np

# Global variable to store selected points in order
selected_points = []
selected_vertices = []

def picking_callback(vis):
    # This function is called when the user picks points
    picked_points_indices = vis.get_picked_points()
    # Since we want to record the points in order, we need to keep track
    # of which points have been added since the last callback
    global selected_points, previous_count, selected_vertices
    picked_points = [p.index for p in picked_points_indices[::-1]]
    
    if len(picked_points) != len(selected_points):
        picked_point = [p for p in picked_points if p not in np.array(selected_vertices)]
        print(picked_points)
        current_count = len(picked_points)
        new_indices = picked_point
    
        # Get the coordinates of the newly selected vertices
        selected_vertices.extend(new_indices)
        new_points = np.asarray(mesh.vertices)[new_indices]
        selected_points.extend(new_points)
        
        if current_count > previous_count:
            print(f"Selected points so far ({len(selected_points)}):")
            for idx, pt in enumerate(selected_points):
                print(f"Point {idx + 1}: {pt}")
            
        previous_count = current_count

gr = 10
mesh = o3d.io.read_triangle_mesh(folder + f'ATLAS_Gr{gr}.ply')
mesh.compute_vertex_normals()

# Create the VisualizerWithEditing instance
vis = o3d.visualization.VisualizerWithVertexSelection()
vis.create_window()
vis.add_geometry(mesh)

# Initialize previous count
previous_count = 0

# Register the picking callback
vis.register_animation_callback(picking_callback)

print("Instructions:")
print(" - Use [shift + left click] to pick points on the mesh.")
print(" - Press 'Q' or close the window when done.")

# Run the visualizer (user picks points here)
vis.run()
vis.destroy_window()

# After the visualizer is closed, selected_points contains the points in order
selected_points = np.array(selected_points)
selected_dict = {idx+1: v.tolist() for idx, v in enumerate(selected_points)}
selected_dict_pinned = {idx+1: v for idx, v in enumerate(selected_vertices)}

print("Final selected landmark points in clicking order:")
for idx, pt in enumerate(selected_points):
    print(f"Point {idx + 1}: {pt}")

[Open3D INFO] Clearing all points from selection.
Instructions:
 - Use [shift + left click] to pick points on the mesh.
 - Press 'Q' or close the window when done.
[Open3D INFO] Adding point #16531 (2.88, 5.71, 112.99) to selection.
[16531]
Selected points so far (1):
Point 1: [  2.877119   5.71351  112.991516]
[Open3D INFO] Adding point #17502 (70.20, 1.47, -83.07) to selection.
[16531, 17502]
Selected points so far (2):
Point 1: [  2.877119   5.71351  112.991516]
Point 2: [ 70.197945   1.474327 -83.06826 ]
[Open3D INFO] Adding point #13805 (43.99, 2.55, 105.65) to selection.
[16531, 17502, 13805]
Selected points so far (3):
Point 1: [  2.877119   5.71351  112.991516]
Point 2: [ 70.197945   1.474327 -83.06826 ]
Point 3: [ 43.993027   2.54799  105.645935]
[Open3D INFO] No point has been picked.
[Open3D INFO] Adding point #12393 (97.66, -10.28, -54.64) to selection.
[16531, 17502, 12393, 13805]
Selected points so far (4):
Point 1: [  2.877119   5.71351  112.991516]
Point 2: [ 70.197945 

In [44]:
# delete p5 to p10 from reference list

reference_list = (
    [
        f'p{i}' for i in range(1, 16)
        if i not in [5, 6, 7, 8, 9, 10, 12, 13, 14]
    ] +
    ['p1p3_0', 'p2p4_0'] +
    [
        f'myo_lm{X}_{suffix}'
        for X in [1, 3] for suffix in ['in', 'out']
    ]
)

print(reference_list)

['p1', 'p2', 'p3', 'p4', 'p11', 'p15', 'p1p3_0', 'p2p4_0', 'myo_lm1_in', 'myo_lm1_out', 'myo_lm3_in', 'myo_lm3_out']


In [45]:
# reference_list = v.myo_myo_landmark_names
selected_dict_aux = {}

# print(f'Landmark names:')
# [print(f'\t{name}') for name in reference_list]

print(len(reference_list), len(selected_dict))
assert len(selected_dict) == len(reference_list), 'Number of landmarks does not match.'

for name in selected_dict.keys():
    # Change the key to the landmark name
    selected_dict_aux[reference_list[int(name)-1]] = selected_dict[name]

print(selected_dict_aux)

with open(folder_out + f'ATLAS_Gr{gr}_key_points.json', 'w') as f:
    json.dump(selected_dict_aux, f)

with open(folder_out + f'ATLAS_Gr{gr}_landmarks.pinned', 'w') as f:
    # json.dump(selected_dict_pinned.values(), f)
    f.write('\n'.join([str(v) for v in selected_dict_pinned.values()]))

12 12
{'p1': [2.877119, 5.71351, 112.991516], 'p2': [70.197945, 1.474327, -83.06826], 'p3': [43.993027, 2.54799, 105.645935], 'p4': [97.658134, -10.284194, -54.636616], 'p11': [-68.526001, -47.208549, -18.695301], 'p15': [29.820005, 15.463331, 26.55056], 'p1p3_0': [47.111763, -30.324663, 121.698402], 'p2p4_0': [104.951881, -25.013901, -69.32711], 'myo_lm1_in': [13.741284, -17.535137, 61.703671], 'myo_lm1_out': [14.801924, -6.557525, 62.923283], 'myo_lm3_in': [15.033282, -16.108511, -16.312777], 'myo_lm3_out': [18.4627, -6.511328, 0.04926]}


In [61]:
# for gr in range(1, 11):
#     land_path = folder_out + f'ATLAS_Gr{gr}_key_points.json'
#     
#     with open(land_path, 'r') as f:
#         landmarks = json.load(f)
#         
#     names = list(landmarks.keys())
#     coords = np.array(list(landmarks.values()))
#     
#     new_landmarks = {}
#     
#     for name, coord in zip(names, coords[::-1]):
#         new_landmarks[name] = coord.tolist()
#         
#     with open(land_path, 'w') as f:
#         json.dump(new_landmarks, f)

TRNASFORM LANDMARKS (COORDS) TO .PINNED FOR EACH EMBRYO

In [54]:
from auxiliary.data.dataset_ht import HtDataset, find_group


ds = HtDataset()
specimens = []
for group_name, group_specimens in ds.specimens.items():
    specimens.extend(group_specimens)
    
    
landmarks_guide_names = v.myo_myo_landmark_names
for i, specimen in enumerate(specimens):
    # if i > 0:
    #     break
    path = v.data_path + f'Landmarks/2019{specimen}_key_points.json'
    with open(path, 'r') as f:
        landmarks = json.load(f)
        
    names = list(landmarks.keys())
    coords = np.array(list(landmarks.values()))
    
    gr = find_group(specimen)
    mesh_path = v.data_path + f'{gr}/3DShape/Tissue/myocardium/2019{specimen}Shape.ply'
    mesh = o3d.io.read_triangle_mesh(mesh_path)
    mesh.compute_vertex_normals()
    
    kdtree = cKDTree(np.asarray(mesh.vertices))
    
    new_landmarks = {}
    
    for name, coord in zip(names, coords[::-1]):
        if name in landmarks_guide_names:
            _, idx = kdtree.query(coord)            
            new_landmarks[name] = idx
            
    print(new_landmarks)
    with open(v.data_path + f'Landmarks/2019{specimen}_landmarks.pinned', 'w') as f:
        f.write('\n'.join([str(v) for v in new_landmarks.values()]))

{'p1': 3317, 'p2': 213, 'p3': 2406, 'p4': 1642, 'p11': 2375, 'p15': 1065, 'p1p3_0': 691, 'p2p4_0': 4251, 'myo_lm1_in': 873, 'myo_lm1_out': 2215, 'myo_lm3_in': 2436, 'myo_lm3_out': 2579}
{'p1': 2294, 'p2': 4888, 'p3': 384, 'p4': 873, 'p11': 1440, 'p15': 1823, 'p1p3_0': 2225, 'p2p4_0': 1774, 'myo_lm1_in': 1057, 'myo_lm1_out': 2543, 'myo_lm3_in': 1953, 'myo_lm3_out': 3395}
{'p1': 1071, 'p2': 3097, 'p3': 860, 'p4': 317, 'p11': 2689, 'p15': 810, 'p1p3_0': 3742, 'p2p4_0': 3956, 'myo_lm1_in': 613, 'myo_lm1_out': 2149, 'myo_lm3_in': 802, 'myo_lm3_out': 168}
{'p1': 414, 'p2': 637, 'p3': 3560, 'p4': 3560, 'p11': 4967, 'p15': 1077, 'p1p3_0': 4920, 'p2p4_0': 3793, 'myo_lm1_in': 975, 'myo_lm1_out': 3472, 'myo_lm3_in': 3333, 'myo_lm3_out': 2221}
{'p1': 4636, 'p2': 4765, 'p3': 4405, 'p4': 4405, 'p11': 3110, 'p15': 376, 'p1p3_0': 1766, 'p2p4_0': 4332, 'myo_lm1_in': 1567, 'myo_lm1_out': 3407, 'myo_lm3_in': 1948, 'myo_lm3_out': 1948}
{'p1': 1513, 'p2': 1513, 'p3': 2577, 'p4': 2577, 'p11': 1821, 'p15': 4